In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Insurance_Multilingual_Policy_Dataset.csv to Insurance_Multilingual_Policy_Dataset.csv


In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [ ]:
insurance_multilingual = pd.read_csv('/content/Insurance_Multilingual_Policy_Dataset.csv')
insurance_multilingual.head()

,Policy_ID,Policy_Text_EN,Policy_Text_FR,Policy_Text_ES,Summarized_Text
0,5ea4d7cd-a834-48b8-9b80-5d5b4e79263b,"Travel insurance covers trip cancellations, lo...",L'assurance voyage couvre les annulations de v...,El seguro de viaje cubre cancelaciones de viaj...,"Covers travel risks such as cancellations, los..."
1,369b1a12-b8fa-4235-8435-b0ff839797db,"Home insurance protects against fire, theft, a...",L'assurance habitation protège contre l'incend...,"El seguro de hogar protege contra incendios, r...",Protects home structure and belongings from ri...
2,9cb6703b-9ef3-4692-ae38-8da75a197760,"Home insurance protects against fire, theft, a...",L'assurance habitation protège contre l'incend...,"El seguro de hogar protege contra incendios, r...",Protects home structure and belongings from ri...
3,9964fb8e-2523-4147-ac66-baf944c88595,Life insurance ensures financial security for ...,L'assurance vie assure la sécurité financière ...,El seguro de vida garantiza la seguridad finan...,Offers financial support for beneficiaries in ...
4,1e4a9661-9d57-472c-ab67-1c4d29f106e1,Health insurance policy provides coverage for ...,La police d'assurance maladie couvre l'hospita...,La póliza de seguro de salud cubre hospitaliza...,Provides medical coverage including hospital v...


In [ ]:
insurance_multilingual.isna().sum()

,0
Policy_ID,0
Policy_Text_EN,0
Policy_Text_FR,0
Policy_Text_ES,0
Summarized_Text,0


In [ ]:
insurance_multilingual.duplicated().sum()

np.int64(0)

In [ ]:
insurance_multilingual.shape

(200, 5)

In [ ]:
!pip install nltk
!pip install spacy
!pip install contractions

In [ ]:
import nltk
nltk.download('stopwords')
!python -m spacy download fr_core_news_sm
!python -m spacy download es_core_news_sm

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 75.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 120.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import re
import unicodedata
from nltk.corpus import stopwords
import spacy
import contractions

# Load language models (for lemmatization)
nlp_fr = spacy.load('fr_core_news_sm', disable=['parser', 'ner'])
nlp_es = spacy.load('es_core_news_sm', disable=['parser', 'ner'])


def expand_english_contractions(text):
    return contractions.fix(text)

def universal_clean(text):
    # Applies to ALL languages
    text = text.lower()
    text = re.sub(r'http\S+|www\S+|@\S+|[^a-zA-Z\s]', '', text)  # Keeps letters/spaces
    return text.strip()

def language_specific_clean(text, lang='en'):
    # Language-specific rules
    if lang == 'en':
        text = expand_english_contractions(text)
        text = ' '.join([word for word in text.split() if word not in stopwords.words('english')])
    elif lang == 'fr':
        text = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('utf-8')  # Remove accents
        text = ' '.join([word for word in text.split() if word not in stopwords.words('french')])
        doc = nlp_fr(text)
        text = ' '.join([token.lemma_ for token in doc])
    elif lang == 'es':
        text = re.sub(r'[áéíóú]', lambda x: x.group(0).translate(str.maketrans('áéíóú', 'aeiou')), text)  # Replace vowels, keep ñ
        text = ' '.join([word for word in text.split() if word not in stopwords.words('spanish')])
        doc = nlp_es(text)
        text = ' '.join([token.lemma_ for token in doc])
    return text

# Apply to DataFrame
insurance_multilingual['Policy_Text_EN'] = insurance_multilingual['Policy_Text_EN'].apply(universal_clean).apply(lambda x: language_specific_clean(x, 'en'))
insurance_multilingual['Policy_Text_FR'] = insurance_multilingual['Policy_Text_FR'].apply(universal_clean).apply(lambda x: language_specific_clean(x, 'fr'))
insurance_multilingual['Policy_Text_ES'] = insurance_multilingual['Policy_Text_ES'].apply(universal_clean).apply(lambda x: language_specific_clean(x, 'es'))

In [ ]:
insurance_multilingual

,Policy_ID,Policy_Text_EN,Policy_Text_FR,Policy_Text_ES,Summarized_Text
0,5ea4d7cd-a834-48b8-9b80-5d5b4e79263b,travel insurance covers trip cancellations los...,lassurance voyag couvr annulatier voyage perte...,seguro viaje cubrir cancelación viaje prdida e...,"Covers travel risks such as cancellations, los..."
1,369b1a12-b8fa-4235-8435-b0ff839797db,home insurance protects fire theft natural dis...,lassurance habitation protge contre lincendie ...,seguro hogar proteger incendio robo desastre n...,Protects home structure and belongings from ri...
2,9cb6703b-9ef3-4692-ae38-8da75a197760,home insurance protects fire theft natural dis...,lassurance habitation protge contre lincendie ...,seguro hogar proteger incendio robo desastre n...,Protects home structure and belongings from ri...
3,9964fb8e-2523-4147-ac66-baf944c88595,life insurance ensures financial security bene...,lassurance vie assurer scurit financire bnfici...,seguro vida garantizar seguridad financiero be...,Offers financial support for beneficiaries in ...
4,1e4a9661-9d57-472c-ab67-1c4d29f106e1,health insurance policy provides coverage hosp...,police dassuranc maladie couvr lhospitalisatio...,plizar seguro salud cubrir hospitalizacin ciru...,Provides medical coverage including hospital v...
...,...,...,...,...,...
195,834d04c4-ea14-44eb-8024-dabb2dcefd20,policy covers vehicle damages due accidents th...,ce police couvr dommage vhicule devoir acciden...,plizar cubrir dao vehculo debido accidente rob...,"Covers vehicle damages, liability, and natural..."
196,95735cda-cce6-4d64-963b-6e4834f5bbda,travel insurance covers trip cancellations los...,lassurance voyag couvr annulatier voyage perte...,seguro viaje cubrir cancelación viaje prdida e...,"Covers travel risks such as cancellations, los..."
197,c73ae02f-f18b-4400-84b7-9b8ccfdfda7d,home insurance protects fire theft natural dis...,lassurance habitation protge contre lincendie ...,seguro hogar proteger incendio robo desastre n...,Protects home structure and belongings from ri...
198,3e289f41-2ba7-46aa-a9d8-a3836e58db21,policy covers vehicle damages due accidents th...,ce police couvr dommage vhicule devoir acciden...,plizar cubrir dao vehculo debido accidente rob...,"Covers vehicle damages, liability, and natural..."


In [ ]:
pip install tokenizers transformers

In [ ]:
from tokenizers import BertWordPieceTokenizer

# Initialize tokenizer
tokenizer = BertWordPieceTokenizer(
    clean_text=True,
    handle_chinese_chars=False,
    strip_accents=False,  # Keep accents for French/Spanish
    lowercase=True
)

# Train on your corpus
files = ["Insurance_Multilingual_Policy_Dataset.csv"]  # File with your text data (one sentence per line)
tokenizer.train(
    files,
    vocab_size=30_000,  # Adjust based on your data size
    min_frequency=2,    # Ignore very rare terms
    special_tokens=["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"]
)

# Save and load
tokenizer.save_model("./", "insurance_tokenizer")

# Apply Hugging Face tokenizer for embeddings using pre-trained BERT tokenizer
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
# Tokenize the text data
def tokenize_text(text, tokenizer):
    return tokenizer.encode(text)

# Apply tokenization to each language column
insurance_multilingual['Policy_Text_EN_tokenized'] = insurance_multilingual['Policy_Text_EN'].apply(lambda x: tokenize_text(x, tokenizer))
insurance_multilingual['Policy_Text_FR_tokenized'] = insurance_multilingual['Policy_Text_FR'].apply(lambda x: tokenize_text(x, tokenizer))
insurance_multilingual['Policy_Text_ES_tokenized'] = insurance_multilingual['Policy_Text_ES'].apply(lambda x: tokenize_text(x, tokenizer))

# Display the DataFrame with tokenized columns
insurance_multilingual


,Policy_ID,Policy_Text_EN,Policy_Text_FR,Policy_Text_ES,Summarized_Text,Policy_Text_EN_tokenized,Policy_Text_FR_tokenized,Policy_Text_ES_tokenized
0,5ea4d7cd-a834-48b8-9b80-5d5b4e79263b,travel insurance covers trip cancellations los...,lassurance voyag couvr annulatier voyage perte...,seguro viaje cubrir cancelación viaje prdida e...,"Covers travel risks such as cancellations, los...","[101, 3604, 5427, 4472, 4440, 16990, 2015, 243...","[101, 27333, 25863, 29536, 3148, 2290, 2522, 2...","[101, 7367, 27390, 2080, 3081, 6460, 21987, 30..."
1,369b1a12-b8fa-4235-8435-b0ff839797db,home insurance protects fire theft natural dis...,lassurance habitation protge contre lincendie ...,seguro hogar proteger incendio robo desastre n...,Protects home structure and belongings from ri...,"[101, 2188, 5427, 18227, 2543, 11933, 3019, 18...","[101, 27333, 25863, 6552, 3258, 4013, 2102, 33...","[101, 7367, 27390, 2080, 27589, 2906, 28567, 2..."
2,9cb6703b-9ef3-4692-ae38-8da75a197760,home insurance protects fire theft natural dis...,lassurance habitation protge contre lincendie ...,seguro hogar proteger incendio robo desastre n...,Protects home structure and belongings from ri...,"[101, 2188, 5427, 18227, 2543, 11933, 3019, 18...","[101, 27333, 25863, 6552, 3258, 4013, 2102, 33...","[101, 7367, 27390, 2080, 27589, 2906, 28567, 2..."
3,9964fb8e-2523-4147-ac66-baf944c88595,life insurance ensures financial security bene...,lassurance vie assurer scurit financire bnfici...,seguro vida garantizar seguridad financiero be...,Offers financial support for beneficiaries in ...,"[101, 2166, 5427, 21312, 3361, 3036, 3841, 128...","[101, 27333, 25863, 20098, 14306, 2099, 8040, ...","[101, 7367, 27390, 2080, 19830, 11721, 17884, ..."
4,1e4a9661-9d57-472c-ab67-1c4d29f106e1,health insurance policy provides coverage hosp...,police dassuranc maladie couvr lhospitalisatio...,plizar seguro salud cubrir hospitalizacin ciru...,Provides medical coverage including hospital v...,"[101, 2740, 5427, 3343, 3640, 6325, 2902, 3989...","[101, 2610, 8695, 26210, 2319, 2278, 28935, 10...","[101, 20228, 21335, 2099, 7367, 27390, 2080, 1..."
...,...,...,...,...,...,...,...,...
195,834d04c4-ea14-44eb-8024-dabb2dcefd20,policy covers vehicle damages due accidents th...,ce police couvr dommage vhicule devoir acciden...,plizar cubrir dao vehculo debido accidente rob...,"Covers vehicle damages, liability, and natural...","[101, 3343, 4472, 4316, 12394, 2349, 13436, 11...","[101, 8292, 2610, 2522, 2226, 19716, 14383, 26...","[101, 20228, 21335, 2099, 21987, 3089, 2099, 4..."
196,95735cda-cce6-4d64-963b-6e4834f5bbda,travel insurance covers trip cancellations los...,lassurance voyag couvr annulatier voyage perte...,seguro viaje cubrir cancelación viaje prdida e...,"Covers travel risks such as cancellations, los...","[101, 3604, 5427, 4472, 4440, 16990, 2015, 243...","[101, 27333, 25863, 29536, 3148, 2290, 2522, 2...","[101, 7367, 27390, 2080, 3081, 6460, 21987, 30..."
197,c73ae02f-f18b-4400-84b7-9b8ccfdfda7d,home insurance protects fire theft natural dis...,lassurance habitation protge contre lincendie ...,seguro hogar proteger incendio robo desastre n...,Protects home structure and belongings from ri...,"[101, 2188, 5427, 18227, 2543, 11933, 3019, 18...","[101, 27333, 25863, 6552, 3258, 4013, 2102, 33...","[101, 7367, 27390, 2080, 27589, 2906, 28567, 2..."
198,3e289f41-2ba7-46aa-a9d8-a3836e58db21,policy covers vehicle damages due accidents th...,ce police couvr dommage vhicule devoir acciden...,plizar cubrir dao vehculo debido accidente rob...,"Covers vehicle damages, liability, and natural...","[101, 3343, 4472, 4316, 12394, 2349, 13436, 11...","[101, 8292, 2610, 2522, 2226, 19716, 14383, 26...","[101, 20228, 21335, 2099, 21987, 3089, 2099, 4..."


In [ ]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
!pip install transformers datasets evaluate
!pip install sentencepiece

In [ ]:
# Translating the text (training mBart model)
from transformers import MarianMTModel, MarianTokenizer
import torch

# Load model and tokenizer ONCE (outside the function)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Function to Translate in Batches
def batch_translate_texts(texts, tokenizer, model, batch_size=16):
    """Translates a batch of texts using the given tokenizer and model."""
    translations = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)  # Move to GPU

        with torch.no_grad():
            translated_tokens = model.generate(**inputs)

        translations.extend([tokenizer.decode(t, skip_special_tokens=True) for t in translated_tokens])
    return translations

# Load Translation Models and Move to GPU
tokenizer_fr = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-fr")
model_fr = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-fr").to(device)  # Move model to GPU

tokenizer_es = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-es")
model_es = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-es").to(device)  # Move model to GPU



In [ ]:

texts = insurance_multilingual['Policy_Text_EN'].fillna("").tolist()  # Replace NaN with empty strings

# French Translation
insurance_multilingual["French_Translation"] = batch_translate_texts(texts,tokenizer_fr, model_fr, batch_size=16)

# Spanish Translation
insurance_multilingual["Spanish_Translation"] = batch_translate_texts(texts,tokenizer_es, model_es, batch_size=16)

insurance_multilingual

,Policy_ID,Policy_Text_EN,Policy_Text_FR,Policy_Text_ES,Summarized_Text,Policy_Text_EN_tokenized,Policy_Text_FR_tokenized,Policy_Text_ES_tokenized,French_Translation,Spanish_Translation
0,5ea4d7cd-a834-48b8-9b80-5d5b4e79263b,travel insurance covers trip cancellations los...,lassurance voyag couvr annulatier voyage perte...,seguro viaje cubrir cancelación viaje prdida e...,"Covers travel risks such as cancellations, los...","[101, 3604, 5427, 4472, 4440, 16990, 2015, 243...","[101, 27333, 25863, 29536, 3148, 2290, 2522, 2...","[101, 7367, 27390, 2080, 3081, 6460, 21987, 30...",assurance voyage couvre les annulations de voy...,seguro de viaje cubre cancelaciones de viaje p...
1,369b1a12-b8fa-4235-8435-b0ff839797db,home insurance protects fire theft natural dis...,lassurance habitation protge contre lincendie ...,seguro hogar proteger incendio robo desastre n...,Protects home structure and belongings from ri...,"[101, 2188, 5427, 18227, 2543, 11933, 3019, 18...","[101, 27333, 25863, 6552, 3258, 4013, 2102, 33...","[101, 7367, 27390, 2080, 27589, 2906, 28567, 2...",l'assurance habitation protège le vol d'incend...,seguro de vivienda protege robo de incendios c...
2,9cb6703b-9ef3-4692-ae38-8da75a197760,home insurance protects fire theft natural dis...,lassurance habitation protge contre lincendie ...,seguro hogar proteger incendio robo desastre n...,Protects home structure and belongings from ri...,"[101, 2188, 5427, 18227, 2543, 11933, 3019, 18...","[101, 27333, 25863, 6552, 3258, 4013, 2102, 33...","[101, 7367, 27390, 2080, 27589, 2906, 28567, 2...",l'assurance habitation protège le vol d'incend...,seguro de vivienda protege robo de incendios c...
3,9964fb8e-2523-4147-ac66-baf944c88595,life insurance ensures financial security bene...,lassurance vie assurer scurit financire bnfici...,seguro vida garantizar seguridad financiero be...,Offers financial support for beneficiaries in ...,"[101, 2166, 5427, 21312, 3361, 3036, 3841, 128...","[101, 27333, 25863, 20098, 14306, 2099, 8040, ...","[101, 7367, 27390, 2080, 19830, 11721, 17884, ...",l'assurance vie assure la sécurité financière ...,seguro de vida asegura beneficiarios de seguri...
4,1e4a9661-9d57-472c-ab67-1c4d29f106e1,health insurance policy provides coverage hosp...,police dassuranc maladie couvr lhospitalisatio...,plizar seguro salud cubrir hospitalizacin ciru...,Provides medical coverage including hospital v...,"[101, 2740, 5427, 3343, 3640, 6325, 2902, 3989...","[101, 2610, 8695, 26210, 2319, 2278, 28935, 10...","[101, 20228, 21335, 2099, 7367, 27390, 2080, 1...",La police d'assurance-maladie prévoit la couve...,póliza de seguro médico proporciona cobertura ...
...,...,...,...,...,...,...,...,...,...,...
195,834d04c4-ea14-44eb-8024-dabb2dcefd20,policy covers vehicle damages due accidents th...,ce police couvr dommage vhicule devoir acciden...,plizar cubrir dao vehculo debido accidente rob...,"Covers vehicle damages, liability, and natural...","[101, 3343, 4472, 4316, 12394, 2349, 13436, 11...","[101, 8292, 2610, 2522, 2226, 19716, 14383, 26...","[101, 20228, 21335, 2099, 21987, 3089, 2099, 4...",la police couvre les dommages causés aux véhic...,póliza cubre daños de vehículos debidos a acci...
196,95735cda-cce6-4d64-963b-6e4834f5bbda,travel insurance covers trip cancellations los...,lassurance voyag couvr annulatier voyage perte...,seguro viaje cubrir cancelación viaje prdida e...,"Covers travel risks such as cancellations, los...","[101, 3604, 5427, 4472, 4440, 16990, 2015, 243...","[101, 27333, 25863, 29536, 3148, 2290, 2522, 2...","[101, 7367, 27390, 2080, 3081, 6460, 21987, 30...",assurance voyage couvre les annulations de voy...,seguro de viaje cubre cancelaciones de viaje p...
197,c73ae02f-f18b-4400-84b7-9b8ccfdfda7d,home insurance protects fire theft natural dis...,lassurance habitation protge contre lincendie ...,seguro hogar proteger incendio robo desastre n...,Protects home structure and belongings from ri...,"[101, 2188, 5427, 18227, 2543, 11933, 3019, 18...","[

In [ ]:
!pip install sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer, util

# Load a multilingual embedding model
embedder = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

# Compute embeddings for all texts
english_embeddings = embedder.encode(insurance_multilingual["Policy_Text_EN"].fillna("").tolist(), convert_to_tensor=True)
french_embeddings = embedder.encode(insurance_multilingual["French_Translation"].fillna("").tolist(), convert_to_tensor=True)
spanish_embeddings = embedder.encode(insurance_multilingual["Spanish_Translation"].fillna("").tolist(), convert_to_tensor=True)

# Compute cosine similarities
cosine_sim_fr = util.pytorch_cos_sim(english_embeddings, french_embeddings).diag()
cosine_sim_es = util.pytorch_cos_sim(english_embeddings, spanish_embeddings).diag()

# Add similarity scores to the DataFrame
insurance_multilingual["Similarity_EN-FR"] = cosine_sim_fr.cpu().numpy()
insurance_multilingual["Similarity_EN-ES"] = cosine_sim_es.cpu().numpy()

# Define the threshold for acceptable similarity
threshold = 0.7

# Compute the percentage of translations above the threshold
fr_similar_count = (insurance_multilingual["Similarity_EN-FR"] >= threshold).sum()
es_similar_count = (insurance_multilingual["Similarity_EN-ES"] >= threshold).sum()

# Compute total rows
total_rows = len(insurance_multilingual)

# Compute percentages
fr_similarity_percent = (fr_similar_count / total_rows) * 100
es_similarity_percent = (es_similar_count / total_rows) * 100

print(f"French Similarity: {fr_similarity_percent:.2f}% of translations have similarity >= {threshold}")
print(f"Spanish Similarity: {es_similarity_percent:.2f}% of translations have similarity >= {threshold}")

French Similarity: 82.50% of translations have similarity >= 0.7
Spanish Similarity: 100.00% of translations have similarity >= 0.7


In [ ]:
insurance_multilingual.to_csv('insurance_processed.csv', index=False)
from google.colab import files
# files.download('insurance_processed.csv')

In [ ]:
pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 10.2 MB/s eta 0:00:00


In [ ]:
!pip install evaluate
!pip install datasets --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.

In [ ]:
pip install --upgrade transformers

In [ ]:
from google.colab import files
uploaded = files.upload()


filename = list(uploaded.keys())[0]
# Load the dataset
df = pd.read_csv(filename)

Saving insurance_processed.csv to insurance_processed.csv


In [ ]:
from datasets import Dataset
import torch

dataset = Dataset.from_pandas(df)
dataset = dataset.train_test_split(test_size=0.1)

print(f"Dataset split: {len(dataset['train'])} train | {len(dataset['test'])} test")

Dataset split: 180 train | 20 test


In [ ]:
from transformers import MarianMTModel, MarianTokenizer

tokenizer_fr = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-fr")
model_fr = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-fr")

tokenizer_es = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-es")
model_es = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-es")


In [ ]:

# 🔹 Step 3: Preprocessing Function
def preprocess_function_fr(examples, tokenizer_fr):

    tokenizer_fr.src_lang = "en_XX"
    tokenizer_fr.tgt_lang = "fr_XX"

    # Prepare the inputs (English)
    inputs_fr = tokenizer_fr(
        examples["Policy_Text_EN"], padding="max_length", truncation=True, max_length=128
    )

    # Select the correct target column based on the language choice (French or Spanish)
    # target_column = f"{target_language}_Translation"
    targets = tokenizer_fr(
        examples["French_Translation"], padding="max_length", truncation=True, max_length=128
    )

    # Set the labels (i.e., the target translation tokens)
    inputs_fr["labels"] = targets["input_ids"]
    return inputs_fr


def preprocess_function_es(examples, tokenizer_es):

    tokenizer_es.src_lang = "en_XX"
    tokenizer_es.tgt_lang = "es_XX"

    # Prepare the inputs (English)
    inputs_es = tokenizer_es(
        examples["Policy_Text_EN"], padding="max_length", truncation=True, max_length=128
    )
    # Select the correct target column based on the language choice (French or Spanish)
    # target_column = f"{target_language}_Translation"
    targets = tokenizer_es(
        examples["Spanish_Translation"], padding="max_length", truncation=True, max_length=128
    )

    # Set the labels (i.e., the target translation tokens)
    inputs_es["labels"] = targets["input_ids"]
    return inputs_es



# 🔹 Step 6: Tokenize Dataset for Both French and Spanish
# Tokenize for French
tokenized_dataset_fr = dataset.map(lambda examples: preprocess_function_fr(examples, tokenizer_fr),
                                    batched=True,
                                    remove_columns=["Policy_Text_EN", "French_Translation", "Spanish_Translation"]
                                )

# Tokenize for Spanish
tokenized_dataset_es = dataset.map(lambda examples: preprocess_function_es(examples, tokenizer_es),
                                    batched=True,
                                    remove_columns=["Policy_Text_EN", "French_Translation", "Spanish_Translation"]
                                )


Map:   0%|          | 0/180 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Map:   0%|          | 0/180 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import DataCollatorForSeq2Seq

use_fp16=torch.cuda.is_available()

# 🔹 Step 4: Define Training Arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./translation_model",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    # gradient_accumulation_steps=8,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=7,
    load_best_model_at_end=True,
    metric_for_best_model="bleu",
    greater_is_better=True,
    predict_with_generate=True,
    fp16 = use_fp16,
    logging_dir="./logs",
    logging_steps=500,
    report_to="none" # Disable reporting (e.g., WandB, TensorBoard)
)



In [ ]:
# 🔹 Step 5: Define Trainer
import evaluate

data_collator_fr = DataCollatorForSeq2Seq(tokenizer_fr, model=model_fr)
bleu = evaluate.load("sacrebleu")

In [ ]:
import numpy as np

def compute_metrics_fr(eval_pred):
    predictions, labels = eval_pred
    labels = np.where(labels != -100, labels, tokenizer_fr.pad_token_id)
    decoded_preds = tokenizer_fr.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer_fr.batch_decode(labels, skip_special_tokens=True)
    decoded_labels = [[label] for label in decoded_labels]

    result = bleu.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": round(result["score"], 2)}

In [ ]:
sample = tokenized_dataset_fr["train"][1]
print("Sample input IDs:", sample["input_ids"])
print("Sample label IDs:", sample["labels"])
print("Decoded input:", tokenizer_fr.decode(sample["input_ids"]))
print("Decoded label:", tokenizer_fr.decode([t for t in sample["labels"] if t != -100]))


Sample input IDs: [490, 3539, 525, 1252, 6534, 41050, 29960, 4424, 7477, 28518, 1509, 34632, 303, 202, 2547, 1265, 2163, 0, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513]
Sample label IDs: [80, 780, 20, 6, 3084, 21, 122, 6544, 10086, 1746, 4994, 567, 8, 806, 27828, 5373, 13, 6

In [ ]:
# training french

trainer_fr = Seq2SeqTrainer(
    model=model_fr,
    args=training_args,
    train_dataset=tokenized_dataset_fr["train"],
    eval_dataset=tokenized_dataset_fr["test"],
    tokenizer=tokenizer_fr,
    data_collator=data_collator_fr,
    compute_metrics=compute_metrics_fr
)

# Train the model on the French dataset
trainer_fr.train()

Epoch,Training Loss,Validation Loss,Bleu
1,No log,0.082345,89.220000
2,No log,0.016687,97.190000
3,No log,0.013425,97.190000
4,No log,0.013170,97.190000
5,No log,0.013095,97.190000
6,No log,0.013013,97.190000
7,No log,0.013052,97.190000


There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.encoder.embed_positions.weight', 'model.decoder.embed_tokens.weight', 'model.decoder.embed_positions.weight', 'lm_head.weight'].


TrainOutput(global_step=315, training_loss=0.18351540943932912, metrics={'train_runtime': 121.9204, 'train_samples_per_second': 10.335, 'train_steps_per_second': 2.584, 'total_flos': 42711939809280.0, 'train_loss': 0.18351540943932912, 'epoch': 7.0})

In [ ]:
save_path = "./translator_finetuned_mbart_fr"

trainer_fr.save_model(save_path)             # saves model, config, and tokenizer files
tokenizer_fr.save_pretrained(save_path)

('./translator_finetuned_mbart_fr/tokenizer_config.json',
 './translator_finetuned_mbart_fr/special_tokens_map.json',
 './translator_finetuned_mbart_fr/vocab.json',
 './translator_finetuned_mbart_fr/source.spm',
 './translator_finetuned_mbart_fr/target.spm',
 './translator_finetuned_mbart_fr/added_tokens.json')

In [ ]:
loaded_tokenizer = MarianTokenizer.from_pretrained("./translator_finetuned_mbart_fr")
loaded_model = MarianMTModel.from_pretrained("./translator_finetuned_mbart_fr")

# Set source language before tokenizing
loaded_tokenizer.src_lang = "en_XX"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_fr.to(device)


text = "Life insurance is something that is useful to us when needed"
inputs = tokenizer_fr(text, return_tensors="pt").to(device)

# Generate translation
outputs = model_fr.generate(**inputs, max_length=128)

# Decode and print
print(tokenizer_fr.decode(outputs[0], skip_special_tokens=True))


L'assurance vie est quelque chose qui nous est utile au besoin


In [ ]:
import shutil
shutil.make_archive("translator_finetuned_mbart_fr", 'zip', "./translator_finetuned_mbart_fr")
files.download("translator_finetuned_mbart_fr.zip")
print(" Model zipped and ready for download!")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

 Model zipped and ready for download!


In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
# 🔹 Step 5: Define Trainer
data_collator_es = DataCollatorForSeq2Seq(tokenizer_es, model=model_es)
bleu = evaluate.load("sacrebleu")

def compute_metrics_es(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer_es.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer_es.batch_decode(labels, skip_special_tokens=True)
    result = bleu.compute(predictions=decoded_preds, references=[[l] for l in decoded_labels])
    return {"bleu": round(result["score"], 2)}

In [ ]:
# training spanish

trainer_es = Seq2SeqTrainer(
    model=model_es,
    args=training_args,
    train_dataset=tokenized_dataset_es["train"],
    eval_dataset=tokenized_dataset_es["test"],
    tokenizer=tokenizer_es,
    data_collator=data_collator_es,
    compute_metrics=compute_metrics_es,
)

trainer_es.train()

Epoch,Training Loss,Validation Loss,Bleu
1,No log,0.166612,70.090000
2,No log,0.007125,100.000000
3,No log,0.004466,98.390000
4,No log,0.000838,100.000000
5,No log,0.000416,100.000000
6,No log,0.000338,100.000000
7,No log,0.000322,100.000000


There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.encoder.embed_positions.weight', 'model.decoder.embed_tokens.weight', 'model.decoder.embed_positions.weight', 'lm_head.weight'].


TrainOutput(global_step=315, training_loss=0.26247030591207837, metrics={'train_runtime': 160.8352, 'train_samples_per_second': 7.834, 'train_steps_per_second': 1.959, 'total_flos': 42711939809280.0, 'train_loss': 0.26247030591207837, 'epoch': 7.0})

In [ ]:
save_path = "./translator_finetuned_mbart_es"

trainer_es.save_model(save_path)             # saves model, config, and tokenizer files
tokenizer_es.save_pretrained(save_path)


('./translator_finetuned_mbart_es/tokenizer_config.json',
 './translator_finetuned_mbart_es/special_tokens_map.json',
 './translator_finetuned_mbart_es/vocab.json',
 './translator_finetuned_mbart_es/source.spm',
 './translator_finetuned_mbart_es/target.spm',
 './translator_finetuned_mbart_es/added_tokens.json')

In [ ]:
loaded_tokenizer = MarianTokenizer.from_pretrained("./translator_finetuned_mbart_es")
loaded_model = MarianMTModel.from_pretrained("./translator_finetuned_mbart_es")

# Set source language before tokenizing
loaded_tokenizer.src_lang = "en_XX"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_es.to(device)


text = "Life insurance is something that is useful to us when needed"
inputs = tokenizer_es(text, return_tensors="pt").to(device)

# Generate translation
outputs = model_es.generate(**inputs, max_length=128)

# Decode and print
print(tokenizer_es.decode(outputs[0], skip_special_tokens=True))

seguro de vida es algo que es util para nosotros cuando es necesario


In [ ]:
import shutil
shutil.make_archive("translator_finetuned_mbart_es", 'zip', "./translator_finetuned_mbart_es")
files.download("translator_finetuned_mbart_es.zip")
print(" Model zipped and ready for download!")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

 Model zipped and ready for download!
